In [1]:
import sys
import os
import numpy as np
from models.convnet import ConvNet1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 200

In [5]:
BASE_DIR = '../data/'
GLOVE_DIR = BASE_DIR + '/glove.6B/'
model_path = './models/cnn1d.h5'

In [6]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [52]:
embeddings_index['tv']

array([-0.33059001,  0.76388001, -0.31174999, -0.34669   , -0.51620001,
        0.07432   , -0.65553999,  0.46752   ,  0.16734999, -0.012482  ,
       -0.45976999,  0.014164  ,  0.89868999, -0.73882002, -0.25885001,
       -0.46661001, -0.08033   ,  1.09829998,  0.18057001,  0.43709999,
       -0.02235   ,  2.35249996,  0.060316  , -0.32949001, -0.085766  ,
        0.21262001, -0.28114   ,  0.24053   ,  0.0063776 , -0.13807   ,
        0.28086001,  0.63009   ,  0.21139   ,  0.088464  , -0.088001  ,
       -0.14092   , -0.10866   , -0.67764002,  0.58658999, -0.30103001,
       -0.094127  ,  0.072383  , -0.16707   ,  0.43601   , -0.76853001,
        0.20130999,  0.86719   , -1.50999999, -0.040861  ,  0.25749001,
        0.63809001, -0.13496999,  0.42894   ,  0.26115   , -0.31821999,
        0.1007    ,  0.37928   ,  0.31981999, -0.30849001,  0.30019   ,
       -0.25192001, -0.76776999, -0.39515999,  0.50958002,  0.32975   ,
       -0.16298001,  0.37228   ,  0.10564   ,  1.00450003, -0.63

In [14]:
# prepare embedding matrix
print('Preparing the Embedding Matrix')
nb_words = len(word_index)
embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Preparing the Embedding Matrix


NameError: name 'word_index' is not defined

In [7]:
text = "French presidential candidate Emmanuel Macrons anti-system angle is a sham | Philippe Marlire"
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)

NameError: name 'Tokenizer' is not defined

In [15]:
import pandas as pd
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

df = pd.read_csv('../data/data.csv')

# Removing NaNs (only 5 rows)
df = df[pd.notnull(df['title'])]

pos = df[df['label'] == 1]
neg = df[df['label'] == 0]

# We have more neg samples than pos, so even things out
neg_sample = neg.sample(len(pos))

# Merging pos and neg
data = pos.append(neg_sample, ignore_index=True)

# Get Train, Test and Valid data ready
X, Y = np.array(data['title']), np.array(data['label'])


BASE_DIR = './data/'
GLOVE_DIR = BASE_DIR + '/glove.6B/'

In [23]:
texts = X.tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 41110 unique tokens.


In [29]:
len(data[0])

1000

In [76]:
text = ["Managed Care Has Trouble Treating AIDS  Patients Say"]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
sequences1 = tokenizer.texts_to_sequences(text)
input_sent = pad_sequences(sequences1, maxlen=MAX_SEQUENCE_LENGTH)

In [74]:
input_sent

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [16]:
model = ConvNet1D(vocab_size=len(embeddings_index), embedding_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)

In [33]:
input_sent

array([[0, 0, 0, ..., 0, 0, 2],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ..., 
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 3],
       [0, 0, 0, ..., 0, 0, 4]], dtype=int32)

In [34]:
from keras.models import load_model
model = load_model('../models/saved/cnn1d.h5')

In [42]:
labels = np_utils.to_categorical(np.asarray(Y))
model.evaluate(data[:1000], labels[:1000])


1000/1000 [==============================] - 9s     


[0.65387739402055745, 0.76800000000000002]

In [94]:
text = ["these photos of older lgbti women celebrate breaking beauty norms"]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
sequences1 = tokenizer.texts_to_sequences(text)
input_sent = pad_sequences(sequences1, maxlen=MAX_SEQUENCE_LENGTH)

In [96]:
model.predict(input_sent)[0][1]

0.6128636

In [61]:
len(data[0])

1000

In [54]:
X[0], data[0]

('23 Life Lessons Cosmo Kramer Taught Us',
 array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0

In [2]:
json_string = model.to_json()

In [3]:
from keras.models import model_from_json
m = model_from_json(json_string)

In [4]:
m.load_weights('models/cnn1d_w.h5')

In [5]:
m.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 1000, 200)     8224800     embedding_input_2[0][0]          
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 996, 128)      128128      embedding_1[0][0]                
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 996, 128)      0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
maxpooling1d_1 (MaxPooling1D)    (None, 199, 128)      0           activation_1[0][0]               
___________________________________________________________________________________________

In [23]:
import string
import re

MATCH_MULTIPLE_SPACES = re.compile("\ {2,}")
VOCABULARY_SIZE = 6500
UNK = "<UNK>"
PAD = "<PAD>"

def clean(text):
    text = text.lower()
    for punctuation in string.punctuation:
        text = text.replace(punctuation, " " + punctuation + " ")
    for i in range(10):
        text = text.replace(str(i), " " + str(i) + " ")
    text = MATCH_MULTIPLE_SPACES.sub(" ", text)
    return "\n".join(line.strip() for line in text.split("\n"))

In [27]:
cleaner =  clean("Hello Workd!").lower().split()
input_sent = pad_sequences(cleaner, maxlen=MAX_SEQUENCE_LENGTH)

ValueError: invalid literal for long() with base 10: 'hello'